<a href="https://colab.research.google.com/github/CSC-312/cogno-1b-0925/blob/main/cogno_1_1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4

## Unsloth
FastModel supports most vision and text models.

In [ ]:
import ast
import pandas as pd

import torch
from datasets import Dataset
from transformers import TextStreamer

from unsloth import FastLanguageModel, FastModel
from unsloth.chat_templates import get_chat_template, train_on_responses_only

from trl import SFTTrainer, SFTConfig

max_seq_length = 2048

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

Add LoRA adapters to finetune a small subset of parameters.

In [ ]:
model = FastModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

## Data
Data preparation and preprocessing.

In [ ]:

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma3",
)

In [ ]:
try:
    df = pd.read_csv('/content/export.csv', on_bad_lines='skip')

    dataset = Dataset.from_pandas(df)
    def filter_nans(example):
        return not pd.isna(example['answer'])

    dataset = dataset.filter(filter_nans)

    print(f"Number of rows after removing bad lines and NaNs: {len(dataset)}")

    print("First 5 rows of the dataset after processing:")
    for i in range(min(5, len(dataset))):
        print(dataset[i])

except Exception as e:
    print(f"An error occurred during pandas loading or conversion: {e}")
    dataset = None

In [ ]:
dataset[100]

Apply the Gemma3 chat template and save output to `text`.

In [ ]:
def formatting_prompts_func(examples):
    convos = []
    for question, answer in zip(examples["question"], examples["answer"]):
        # Format the question and answer into the Gemma3 chat template
        messages = [
            {"role": "user", "content": question},
            {"role": "model", "content": answer}
        ]
        convos.append(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False).removeprefix('<bos>'))

    return { "text" : convos, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Preview formatted examples.

In [ ]:
dataset[100]['text']

## Training
Quick example: 100 steps (adjust `num_train_epochs` / `max_steps` for full runs).

In [ ]:

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 20, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 5e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir="outputs",
        report_to = "none", # Use this for WandB etc
    )
)

Use Unsloth's `train_on_responses_only` to mask input loss and focus on assistant outputs.

In [ ]:

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Verify masking by inspecting the 100th example.

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

Masked example — only the answer should appear.

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## Inference
Recommended: temperature=1.0, top_p=0.95, top_k=64.

In [ ]:
messages = [
    {"role" : 'user', 'content' : dataset['text'][44646].split('<start_of_turn>model\n')[0].replace('<start_of_turn>user\n', '').replace('<end_of_turn>\n', '')}
]

# Print the original question
print("Question:", messages[0]['content'])

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
).removeprefix('<bos>')

_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 125,
    temperature = 1, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

## Saving
Save LoRA adapters with `save_pretrained` or `push_to_hub` (saves adapters only).

In [ ]:
model.save_pretrained("cogno-1-270m")  # Local saving
tokenizer.save_pretrained("cogno-1-270m")
# model.push_to_hub("your_name/cogno-1-270m", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/cogno-1-270m", token = "...") # Online saving

Load saved LoRA adapters for inference by calling `from_pretrained`.

In [ ]:
if True:

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "cogno-1-270m".
        max_seq_length = 2048,
        load_in_4bit = False,
    )

## Save float16 (VLLM)
Use `merged_16bit` (float16) or `merged_4bit` (int4). Use `push_to_hub_merged` to upload.

In [ ]:

# Just LoRA adapters
if False:
    model.save_pretrained("cogno-1-270m")
    tokenizer.save_pretrained("cogno-1-270m")
if False: # Pushing to HF Hub
    model.push_to_hub("hf/cogno-1-270m", token = "")
    tokenizer.push_to_hub("hf/cogno-1-270m", token = "")


## GGUF / llama.cpp conversion

In [ ]:
if True: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "cogno-1-270m",
        tokenizer,
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

To upload GGUF to Hugging Face, set the upload flag and provide your token.

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "cogno-1-270m",
        tokenizer,
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/cogno-1-270m-gguf",
        token = "hf_...",
    )



<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
</div>
